In [ ]:
!pip install tensorflow_ranking
!pip install tf2onnx

In [2]:
from tensorflow import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Reshape, Activation, Flatten, Dropout, Conv2D, MaxPooling2D
import tensorflow as tf
import tensorflow_ranking as tfr
import tf2onnx

In [3]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
output_path = 'drive/MyDrive/CURIS/networks/'

In [5]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

11490434/11490434 [==============================] - 0s 0us/step


In [6]:
def eval_and_save(model):
  score = model.evaluate(x_test, y_test, verbose=0)
  print("Test loss:", score[0])
  print("Test accuracy:", score[1])
  model.summary()
  model.save(output_path + model.name + '.h5')
  model_proto, _ = tf2onnx.convert.from_keras(model, output_path=output_path + model.name + '.onnx')

10x2 fully connected

In [7]:
model = Sequential(name='mnist-10x2')
model.add(Flatten(name='input'))
model.add(Dense(10, name='dense_1', activation='relu'))
model.add(Dense(10, name='dense_2', activation='relu'))
model.add(Dense(10, name='logit'))
# model.add(Activation('softmax', name='output'))

In [8]:
model.compile(# loss='categorical_crossentropy',
              # loss=tf.keras.losses.CategoricalCrossentropy(),
              loss=tfr.keras.losses.SoftmaxLoss(),
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])
model.fit(x_train, y_train,
          batch_size=128,
          epochs=20,
          verbose=1,
          validation_data=(x_test, y_test))

Epoch 1/20
469/469 [==============================] - 6s 9ms/step - loss: 0.9163 - accuracy: 0.7025 - val_loss: 0.4314 - val_accuracy: 0.8783
Epoch 2/20
469/469 [==============================] - 5s 10ms/step - loss: 0.3732 - accuracy: 0.8929 - val_loss: 0.3270 - val_accuracy: 0.9075
Epoch 3/20
469/469 [==============================] - 3s 7ms/step - loss: 0.3047 - accuracy: 0.9132 - val_loss: 0.2873 - val_accuracy: 0.9190
Epoch 4/20
469/469 [==============================] - 3s 6ms/step - loss: 0.2724 - accuracy: 0.9226 - val_loss: 0.2682 - val_accuracy: 0.9243
Epoch 5/20
469/469 [==============================] - 3s 6ms/step - loss: 0.2530 - accuracy: 0.9275 - val_loss: 0.2487 - val_accuracy: 0.9305
Epoch 6/20
469/469 [==============================] - 2s 4ms/step - loss: 0.2400 - accuracy: 0.9316 - val_loss: 0.2462 - val_accuracy: 0.9299
Epoch 7/20
469/469 [==============================] - 2s 5ms/step - loss: 0.2301 - accuracy: 0.9350 - val_loss: 0.2307 - val_accuracy: 0.9351
Epoch

In [9]:
eval_and_save(model)

Test loss: 0.207551509141922
Test accuracy: 0.9404000043869019
Model: "mnist-10x2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (Flatten)             (None, 784)               0         
                                                                 
 dense_1 (Dense)             (None, 10)                7850      
                                                                 
 dense_2 (Dense)             (None, 10)                110       
                                                                 
 logit (Dense)               (None, 10)                110       
                                                                 
Total params: 8,070
Trainable params: 8,070
Non-trainable params: 0
_________________________________________________________________


In [10]:
import torch
import torch.nn as nn
from torchvision.datasets import MNIST